# Árvores II - Tarefa 2

### 1. Carregar as bases

Vamos carregar as bases lidas na tarefa passada. Se você salvou essas bases em arquivo texto, basta fazer a leitura com o comando ```pd.read_csv``` das seguintes bases:

- X_train
- Y_train
- X_test
- Y_test

Não se esqueça de considerar a leitura dos índices dos arquivos no ```read_csv()```!

In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report

In [2]:
X_train = pd.read_csv('../../../../Datasets/Human Activity Recognition Using Smartphones/UCI HAR Dataset/train/X_train.csv', sep = ',', index_col=[0, 1])
X_train

In [11]:
Y_train = pd.read_csv('../../../../Datasets/Human Activity Recognition Using Smartphones/UCI HAR Dataset/train/y_train.txt', sep = '\s+', header=None)
# renomeando a coluna para target
Y_train.columns = ['target']
Y_train

### 2. Divisão da base em Treino, Validação e Teste

A base já se encontra dividida em Treino e Validação. O que vamos fazer então é extrair uma base de Validação da base de Treino.

Extraia 25% da base de treino como base de validação.

In [12]:
# train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=42)


### 3. Melhores 3 variáveis

Rode uma árvore com profundidade máxima igual a 4 para prever a atividade humana com todas as variáveis.
Observe a importância das variáveis e considere as 3 variáveis com maior importância para os próximos passos.
Dica: utilize o atributo ```clf.feature_importances_``` da árvore treinada.

In [13]:
tree = DecisionTreeClassifier(max_depth=4, random_state=42)
tree.fit(X_train, Y_train)
best_features = pd.Series(tree.feature_importances_, index=X_train.columns).sort_values(ascending=False).head(3)
best_features

### 4. Construa uma árvore com as 3 melhores variáveis

Utilizando as três variáveis encontradas acima, construa uma árvore de decisão. Encontre o melhor ```ccp_alpha``` utilizando a base de validação, conforme a estrutura que vimos em aula.

In [14]:
%%time
tree = DecisionTreeClassifier(max_depth=4, random_state=42)
path = tree.cost_complexity_pruning_path(X_train[best_features.index], Y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=42, ccp_alpha=ccp_alpha)
    clf.fit(X_train[best_features.index], Y_train)
    clfs.append(clf)


train_scores = [clf.score(X_train[best_features.index], Y_train) for clf in clfs]
val_scores = [clf.score(X_val[best_features.index], Y_val) for clf in clfs]

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and validation sets")
ax.plot(ccp_alphas, train_scores, marker='o', label="train",
        drawstyle="steps-post")
ax.plot(ccp_alphas, val_scores, marker='o', label="validation",
        drawstyle="steps-post")
ax.legend()
plt.show()

In [15]:
print(f'Melhor ccp_alpha: {ccp_alphas[np.argmax(val_scores)]}')
print(f'Melhor acurácia: {max(val_scores)}')

    ### 5. Avaliação do modelo

Avalie a árvore encontrada no item anterior na base de testes.

In [23]:
X_test = pd.read_csv('../../../../Datasets/Human Activity Recognition Using Smartphones/UCI HAR Dataset/test/X_test.csv', sep = ',', index_col=[0, 1])
y_test = pd.read_csv('../../../../Datasets/Human Activity Recognition Using Smartphones/UCI HAR Dataset/test/y_test.txt', sep = '\s+', header=None)
y_test.columns = ['target']

tree = DecisionTreeClassifier(random_state=42, ccp_alpha=ccp_alphas[np.argmax(val_scores)])
tree.fit(X_train[best_features.index], Y_train)
y_pred = tree.predict(X_test[best_features.index])

cm = confusion_matrix(y_test, y_pred)   
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=tree.classes_)
# mudar cores
disp.plot(cmap='Blues')
plt.show()


In [29]:
target_names = [str(cls) for cls in tree.classes_]
print(classification_report(y_test, y_pred, target_names= target_names))


### 6. Conclusão

Acurácia do modelo de treino foi de 0.88 enquanto na base de testes fooi de 0.67 o que indica que o modelo está com overfitting. A matriz de confusão mostra que o modelo está com dificuldade em classificar as atividades de 1 a 5.